In [1]:
%matplotlib inline
from fastai import *
from fastai.vision import *

In [2]:
sz = 128

path = Path('data')
path_img = path/f'train-{sz}'

In [3]:
path_test = path/f'test-{sz}'

In [4]:
list = os.listdir(path_test)
number_files = len(list)
print (number_files)

FileNotFoundError: [Errno 2] No such file or directory: 'data/test-128'

In [ ]:
os.chdir('/home/paperspace')

In [ ]:
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
pat = r'/([^/]+)_\d+.png$'

In [ ]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=128, test=Path('../test-128'))
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)
len(data.classes),data.c

In [ ]:
data.c

## Training: resnet34

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
torch.cuda.device_count()

In [ ]:
learn.save('stage-1')

## Results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

## Unfreezing, fine-tuning, and learning rates

In [ ]:
learn.load('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(5e-6,5e-4))

In [ ]:
learn.save('stage-2')

## Generate submission

In [ ]:
learn.load('stage-2')

In [ ]:
preds = learn.TTA(ds_type = DatasetType.Test)[0]

In [ ]:
len(preds)

In [ ]:
top_3 = np.argsort(preds.numpy())[:, ::-1][:, :3]

In [ ]:
n, _ = top_3.shape
labels = []
for i in range(n):
    labels.append(' '.join([learn.data.train_ds.ds.classes[idx] for idx in top_3[i]]))

In [ ]:
img = learn.data.test_dl.dl.dataset[10][0]

In [ ]:
img

In [ ]:
labels[10]

In [ ]:
key_ids = [path.stem for path in learn.data.test_dl.dl.dataset.x]

In [ ]:
os.makedirs(f'subs', exist_ok=True)

In [ ]:
len(key_ids)

In [ ]:
sub = pd.DataFrame({'key_id': key_ids, 'word': labels})

In [ ]:
sub.head()

In [ ]:
name = 'first_sub'

In [ ]:
sub.to_csv(f'subs/{name}.csv.gz', index=False, compression='gzip')

In [ ]:
pd.read_csv(f'subs/{name}.csv.gz').head()

In [ ]:
!kaggle competitions submit -c quickdraw-doodle-recognition -f subs/{name}.csv.gz -m "{name}"